# Learning Goals

## Optimizing Hugging Face Models with Supervised Fine-Tuning (SFT) in NeMo 2.0

NeMo 2.0 allows users to perform Supervised Fine-Tuning (SFT) and Parameter Efficient Fine-Tuning (PEFT) using Hugging Face (HF) LLMs. NeMo 2.0 utilizes HF's auto classes to download and load HF's transformer models, and wraps these models to turn them into lightning modules in order to perform tasks such as SFT and PEFT with NeMo 2.0. The goal of this feature is to provide day 0 support for the models available in HF.

[AutoModel](https://huggingface.co/docs/transformers/en/model_doc/auto) is the generic model class that will be instantiated as one of the model classes of the library when created with the from_pretrained() class method. There are many AutoModel classes in HF and each of them covers a specific group of transformer model architectures. AutoModel class loads mainly the base transformer model that converts embeddings to hidden states where a specific AutoModel class such as AutoModelForCausalLM has a causal language modeling head on top of the base model.

NeMo 2.0 includes wrapper classes for these auto model classes in HF in order to make them runnable in NeMo pretraining, SFT and PEFT workflows by turning them into a lightning module. Since there are many auto model classes, the widely used auto classes are currently added into the NeMo 2.0. 

In this notebook, we will demonstrate a SFT training example on how to perform SFT with Hugging Face LLMs to make the models more performant on a specific task. We will focus on the models that can be loaded using the HF's `AutoModelForCausalLM` class.

## Data
We will use [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) dataset which is a reading comprehension dataset, consisting of questions and answers pairs.

## Step 1. Import Modules and Prepare the Dataset

In [1]:
import tempfile
from functools import partial

import fiddle as fdl
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader

from nemo import lightning as nl
from nemo.collections import llm
from nemo.lightning.pytorch.callbacks import JitConfig, JitTransform

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[WARNING  | bitsandbytes.cextension]: Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda128.so')
[WARNING  | bitsandbytes.cextension]: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo W 2025-02-03 17:53:41 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pyannote/core/notebook.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
      cm = get_cmap("Set1")
    


We will be using SquadDataModule that NeMo 2.0. provides. This data module extends the `FineTuningDataModule`, so that it has access to existing data handling logic including the packed sequences. 

In [2]:
class SquadDataModuleWithPthDataloader(llm.SquadDataModule):
    """Creates a squad dataset with a PT dataloader"""

    def _create_dataloader(self, dataset, mode, **kwargs) -> DataLoader:
        return DataLoader(
            dataset,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
            persistent_workers=self.persistent_workers,
            collate_fn=dataset.collate_fn,
            batch_size=self.micro_batch_size,
            **kwargs,
        )


def squad(tokenizer, mbs=1, gbs=2) -> pl.LightningDataModule:
    """Instantiates a SquadDataModuleWithPthDataloader and return it

    Args:
        tokenizer (AutoTokenizer): the tokenizer to use

    Returns:
        pl.LightningDataModule: the dataset to train with.
    """
    return SquadDataModuleWithPthDataloader(
        tokenizer=tokenizer,
        seq_length=512,
        micro_batch_size=mbs,
        global_batch_size=gbs,
        num_workers=0,
        dataset_kwargs={
            "sanity_check_dist_workers": False,
            "get_attention_mask_from_fusion": True,
        },
    )

## Step 2. Set Parameters and Start the SFT using a HF Model

Now, we will set some of the important variables including the HF model name, maximum steps, number of GPUs, etc. You can find the details of these parameters below.
- `model_name`: pre-trained HF model or path of a HF model.
- `strategy`: distributed training strategy such as DDP, FSDP, etc. 
- `devices`: number of GPUs to be used in the training.
- `max_steps`: number of steps in the training.
- `wandb_project`: wandb project.
- `use_torch_jit`: enable torch jit or not.
- `ckpt_folder`: path for the checkpoins.

All the popular models such as Llama, GPT, Gemma, Mistral, Phi, and Qwen are supported. After running this workflow, please go ahead and get yourself another HF model name and rerun the notebook with that model. Just please make sure you pick a model that can fit into your GPU(s) memory.

In [3]:
# In order to use the models like Llama, Gemma, you need to ask for permission on the HF model page and then pass the HF_TOKEN in the next cell.
# model_name = "google/gemma-2b" # HF model name. This can be the path of the downloaded model as well.
model_name = "meta-llama/Llama-3.2-1B"  # HF model name. This can be the path of the downloaded model as well.
strategy = "auto" # Distributed training strategy such as DDP, FSDP2, etc.
devices = 1 # Number of GPUs.
max_steps = 100 # Number of steps in the training loop.
accelerator = "gpu"
wandb_project = None
use_torch_jit = False # torch jit can be enabled.
ckpt_folder="/opt/checkpoints/automodel_experiments/" # Path for saving the checkpoint.

Some models have gated access and if you running one of those models, you will need to get access first and then you need to set your HF Token by running the cell below.

In [4]:
import os
os.environ["HF_TOKEN"] ='<HF_TOKEN>'

After setting some of the parameters, we can start the SFT training workflow. Even though the SFT workflow with the HF models/checkpoints seems slightly different than the workflows with NeMo models/checkpoints, we still use the same NeMo 2.0 API. The main difference is the model we pass into the `finetune` API. 



In [ ]:
wandb = None
if wandb_project is not None:
    model = '_'.join(args.model.split('/')[-2:])
    wandb = WandbLogger(
        project=wandb_project,
        name=f'{model}_dev{devices}_strat_{strategy}',
    )

callbacks = []
if use_torch_jit:
    jit_config = JitConfig(use_torch=True, torch_kwargs={'dynamic': False}, use_thunder=False)
    callbacks = [JitTransform(jit_config)]

callbacks.append(
    nl.ModelCheckpoint(
        every_n_train_steps=max_steps // 2,
        dirpath=ckpt_folder,
    )
)

if strategy == 'fsdp2':
    astrategy = nl.FSDP2Strategy(data_parallel_size=devices, tensor_parallel_size=1)

if __name__ == '__main__':
    llm.api.finetune(
        model=llm.HFAutoModelForCausalLM(model_name=model_name),
        data=squad(llm.HFAutoModelForCausalLM.configure_tokenizer(model_name), gbs=devices),
        trainer=nl.Trainer(
            devices=devices,
            max_steps=max_steps,
            accelerator="gpu",
            strategy=strategy,
            log_every_n_steps=1,
            limit_val_batches=0.0,
            num_sanity_val_steps=0,
            accumulate_grad_batches=1,
            gradient_clip_val=1.0,
            use_distributed_sampler=False,
            logger=wandb,
            callbacks=callbacks,
            precision="bf16",
        ),
        optim=fdl.build(llm.adam.pytorch_adam_with_flat_lr(lr=1e-5)),
        log=None,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[INFO     | pytorch_lightning.utilities.rank_zero]: Using bfloat16 Automatic Mixed Precision (AMP)
[INFO     | pytorch_lightning.utilities.rank_zero]: GPU available: True (cuda), used: True
[INFO     | pytorch_lightning.utilities.rank_zero]: TPU available: False, using: 0 TPU cores
[INFO     | pytorch_lightning.utilities.rank_zero]: HPU available: False, using: 0 HPUs


[NeMo I 2025-02-03 17:53:43 nemo_logging:393] Experiments will be logged at /opt/NeMo/tutorials/llm/hf/nemo2_sft/nemo_experiments/default/2025-02-03_17-53-43


[NeMo W 2025-02-03 17:53:43 nemo_logging:405] "update_logger_directory" is True. Overwriting tensorboard logger "save_dir" to /opt/NeMo/tutorials/llm/hf/nemo2_sft/nemo_experiments
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | LlamaForCausalLM | 1.2 B  | train
---------------------------------------------------
1.2 B     Trainable params
0         Non-trainable params
1.2 B     Total params
4,943.258 Total estimated model params size (MB)
231       Modules in train mode
0         Modules in eval mode


[NeMo I 2025-02-03 17:53:47 nemo_logging:393] Building data files
[NeMo I 2025-02-03 17:53:47 nemo_logging:393] Processing 1 data files using 1 workers


[rank: 0] Received SIGTERM: 15


...

Epoch 0:  99%|█████████████████████████████████████████████████████████████▍| 100/101 [00:19<00:00,  5.26it/s, v_num=3-43, train_loss_step=0.0838]

[INFO     | pytorch_lightning.utilities.rank_zero]: Epoch 0, global step 100: 'val_loss' was not in top 3


Epoch 0:  99%|████████████████████████████████████▋| 100/101 [00:24<00:00,  4.12it/s, v_num=3-43, train_loss_step=0.0838, train_loss_epoch=0.0236]

[INFO     | pytorch_lightning.utilities.rank_zero]: `Trainer.fit` stopped: `max_steps=100` reached.


Epoch 0:  99%|████████████████████████████████████▋| 100/101 [00:24<00:00,  4.12it/s, v_num=3-43, train_loss_step=0.0838, train_loss_epoch=0.0236]
